In [1]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git@nightly git+https://github.com/unslothai/unsloth-zoo.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-04-22 18:47:03.045930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745347623.502286      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745347623.615477      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the `Llama-3.1` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. But we convert it to HuggingFace's normal multiturn format `("role", "content")` instead of `("from", "value")`/ Llama-3 renders multi turn conversations like below:

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3` and more.

In [6]:
from datasets import load_dataset
dataset = load_dataset("AntonTi0108/butterboard_syst_analyst_interviews_dataset")

Repo card metadata block was not found. Setting CardData to empty.


In [14]:
dataset['train'][0]

{'instruction': 'Представь, что ты виртуальный наставник. Определи сильные и слабые стороны кандидата по диалогу и сформулируй рекомендации.',
 'input': 'Ответ: Так, да, привет ещё раз. Расскажи в целом… Ну, так, объясню, наверное, у нас сейчас будет собес. Вот мы пробежимся по темам некоторым, которые участвуют у нас в грейдах, в точке в целом. Поговорим про инфраструктуру и так далее. Вот. Расскажи в целом про команду, про проекты. Какую роль ты занимал в команде? Может, что-то интересное вспомнишь? А, ещё раз всем привет! Да, в основном у меня опыт был в интехе.\nВопрос: Три-четыре года был WebMoney опыт. Касаемо проектов, самые интересные это были C2C, то есть у нас была внутренняя валюта в рамках компании, и её нужно было переводить, там, к примеру, в доллары, в рубли, то есть некоторая биржевая, так сказать, Некоторые биржи. Вот. Этот проект у нас был на Monolith. В основном из стека у нас был ASP.NET Core. Последняя версия — это был на тот момент.NET 6-й. Потом мы его тоже переп

In [16]:
import json
from unsloth.chat_templates import get_chat_template


SYSTEM_PROMPT = (
    "Ты — виртуальный наставник, который анализирует диалог кандидата на позицию системного аналитика и интервьюера. "
    "На основании диалога определи сильные и слабые стороны кандидата и сформулируй персонализированные рекомендации. "
    "Исправляй грамматические ошибки и неточности в названиях. Если роли неправильно назначены, "
    "то твоя задача понять из контекста кто является кандидатом и сформулировать для него рекомендации. "
    "Важно: верни строго валидный JSON, содержащий ровно одно поле 'output'. "
    "Внутри 'output' укажи:\n"
    "- hard_skills: [список строк]\n"
    "- soft_skills: [список строк]\n"
    "- recommendations: [список строк]\n"
    "Не добавляй никаких других полей и не используй Markdown‑блоки."
)

def to_chat_format(example):
    system_msg = {"role": "system", "content": SYSTEM_PROMPT}
    user_msg   = {"role": "user",   "content": example["input"]}
    assistant_msg = {
        "role": "assistant",
        "content": json.dumps({"output": example["output"]}, ensure_ascii=False)
    }
    return {"messages": [system_msg, user_msg, assistant_msg]}

dataset2 = dataset.map(
    to_chat_format,
    remove_columns=["instruction", "input", "output", "chunk_id", "source_file"],
    batched=False
)

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [18]:
dataset2['train'][0]

{'messages': [{'content': "Ты — виртуальный наставник, который анализирует диалог кандидата на позицию системного аналитика и интервьюера. На основании диалога определи сильные и слабые стороны кандидата и сформулируй персонализированные рекомендации. Исправляй грамматические ошибки и неточности в названиях. Если роли неправильно назначены, то твоя задача понять из контекста кто является кандидатом и сформулировать для него рекомендации. Важно: верни строго валидный JSON, содержащий ровно одно поле 'output'. Внутри 'output' укажи:\n- hard_skills: [список строк]\n- soft_skills: [список строк]\n- recommendations: [список строк]\nНе добавляй никаких других полей и не используй Markdown‑блоки.",
   'role': 'system'},
  {'content': 'Ответ: Так, да, привет ещё раз. Расскажи в целом… Ну, так, объясню, наверное, у нас сейчас будет собес. Вот мы пробежимся по темам некоторым, которые участвуют у нас в грейдах, в точке в целом. Поговорим про инфраструктуру и так далее. Вот. Расскажи в целом про 

In [20]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


dataset3 = dataset2.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [27]:
dataset3['train']['text'][0]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы — виртуальный наставник, который анализирует диалог кандидата на позицию системного аналитика и интервьюера. На основании диалога определи сильные и слабые стороны кандидата и сформулируй персонализированные рекомендации. Исправляй грамматические ошибки и неточности в названиях. Если роли неправильно назначены, то твоя задача понять из контекста кто является кандидатом и сформулировать для него рекомендации. Важно: верни строго валидный JSON, содержащий ровно одно поле \'output\'. Внутри \'output\' укажи:\n- hard_skills: [список строк]\n- soft_skills: [список строк]\n- recommendations: [список строк]\nНе добавляй никаких других полей и не используй Markdown‑блоки.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nОтвет: Так, да, привет ещё раз. Расскажи в целом… Ну, так, объясню, наверное, у нас сейчас будет собес. Вот мы пробежимся по темам некоторым

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [31]:
dataset3

DatasetDict({
    train: Dataset({
        features: ['messages', 'text'],
        num_rows: 113
    })
})

In [32]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset3['train'],  # или dataset2, в зависимости от того, где у вас текст
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size     = 2,
        gradient_accumulation_steps     = 4,
        warmup_steps                    = 5,
        num_train_epochs                = 3,       # <-- три эпохи
        # убрали max_steps, чтобы не рубить посередине
        learning_rate                   = 2e-4,
        fp16                            = not is_bfloat16_supported(),
        bf16                            = is_bfloat16_supported(),
        logging_steps                   = 1,
        optim                           = "adamw_8bit",
        weight_decay                    = 0.01,
        lr_scheduler_type               = "linear",
        seed                            = 3407,
        output_dir                      = "outputs",
        report_to                       = "none",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/113 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [33]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=4):   0%|          | 0/113 [00:00<?, ? examples/s]

We verify masking is actually done:

In [34]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы — виртуальный наставник, который анализирует диалог кандидата на позицию системного аналитика и интервьюера. На основании диалога определи сильные и слабые стороны кандидата и сформулируй персонализированные рекомендации. Исправляй грамматические ошибки и неточности в названиях. Если роли неправильно назначены, то твоя задача понять из контекста кто является кандидатом и сформулировать для него рекомендации. Важно: верни строго валидный JSON, содержащий ровно одно поле \'output\'. Внутри \'output\' укажи:\n- hard_skills: [список строк]\n- soft_skills: [список строк]\n- recommendations: [список строк]\nНе добавляй никаких других полей и не используй Markdown‑блоки.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nОтвет: А строки приходилось лачить?\nВопрос: Вроде бы за практику было, я вот такое вспоминаю. Селект фор апдейт. Если это 

In [35]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

'                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

We can see the System and Instruction prompts are successfully masked!

In [36]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
3.441 GB of memory reserved.


In [37]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 113 | Num Epochs = 3 | Total steps = 21
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.635300
2,1.501800
3,1.641300
4,1.477500
5,1.387900
6,1.439900
7,1.256400
8,1.177800
9,1.163100
10,1.124700


In [38]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

527.2802 seconds used for training.
8.79 minutes used for training.
Peak reserved memory = 4.33 GB.
Peak reserved memory for training = 0.889 GB.
Peak reserved memory % of max memory = 29.374 %.
Peak reserved memory for training % of max memory = 6.031 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [40]:
dataset['train'][0]

{'instruction': 'Представь, что ты виртуальный наставник. Определи сильные и слабые стороны кандидата по диалогу и сформулируй рекомендации.',
 'input': 'Ответ: Так, да, привет ещё раз. Расскажи в целом… Ну, так, объясню, наверное, у нас сейчас будет собес. Вот мы пробежимся по темам некоторым, которые участвуют у нас в грейдах, в точке в целом. Поговорим про инфраструктуру и так далее. Вот. Расскажи в целом про команду, про проекты. Какую роль ты занимал в команде? Может, что-то интересное вспомнишь? А, ещё раз всем привет! Да, в основном у меня опыт был в интехе.\nВопрос: Три-четыре года был WebMoney опыт. Касаемо проектов, самые интересные это были C2C, то есть у нас была внутренняя валюта в рамках компании, и её нужно было переводить, там, к примеру, в доллары, в рубли, то есть некоторая биржевая, так сказать, Некоторые биржи. Вот. Этот проект у нас был на Monolith. В основном из стека у нас был ASP.NET Core. Последняя версия — это был на тот момент.NET 6-й. Потом мы его тоже переп

In [45]:
from unsloth.chat_templates import get_chat_template

SYSTEM_PROMPT = (
"Ты — виртуальный наставник, который анализирует диалог кандидата на позицию системного аналитика и интервьюера. "
"На основании диалога определи сильные и слабые стороны кандидата и сформулируй "
"персонализированные рекомендации. Исправляй грамматические ошибки и неточности в названиях. "
"Если роли неправильно назначены, то твоя задача понять из контекста кто является кандидатом "
"и сформулировать для него персонализированные рекомендации. "
"Важно: верни строго валидный JSON, содержащий ровно одно поле 'output'. "
"Внутри 'output' укажи:\n"
"- hard_skills: [список строк]\n"
"- soft_skills: [список строк]\n"
"- recommendations: [список строк]\n"
"Не добавляй никаких других полей и не используй Markdown-блоки. "
"Пример корректной структуры:\n"
"{\n"
"  \"output\": {\n"
"    \"hard_skills\": [\"...\"],\n"
"    \"soft_skills\": [\"...\"],\n"
"    \"recommendations\": [\"...\"]\n"
"  }\n"
"}"
)
example = dataset['train'][0]
user_content = f"{example['instruction']}\n\n{example['input']}"

messages = [
    {"role": "system",    "content": SYSTEM_PROMPT},
    {"role": "user",      "content": user_content},
]

tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.1",
)

FastLanguageModel.for_inference(model)

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nТы — виртуальный наставник, который анализирует диалог кандидата на позицию системного аналитика и интервьюера. На основании диалога определи сильные и слабые стороны кандидата и сформулируй персонализированные рекомендации. Исправляй грамматические ошибки и неточности в названиях. Если роли неправильно назначены, то твоя задача понять из контекста кто является кандидатом и сформулировать для него персонализированные рекомендации. Важно: верни строго валидный JSON, содержащий ровно одно поле \'output\'. Внутри \'output\' укажи:\n- hard_skills: [список строк]\n- soft_skills: [список строк]\n- recommendations: [список строк]\nНе добавляй никаких других полей и не используй Markdown-блоки. Пример корректной структуры:\n{\n  "output": {\n    "hard_skills": ["..."],\n    "soft_skills": ["..."],\n    "recommendations": ["..."]\n  }\n}<|eot_id|><|start_header_id|

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [46]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [48]:
model.push_to_hub("AntonTi0108/lora_model_personalization", token = "") # Online saving
tokenizer.push_to_hub("AntonTi0108/lora_model_personalization", token = "") # Online saving


README.md:   0%|          | 0.00/600 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

Saved model to https://huggingface.co/AntonTi0108/lora_model_personalization


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Describe a tall tower in the capital of France."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # Get a token at https://huggingface.co/settings/tokens
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)
9. [**NEW**] We make Phi-3 Medium / Mini **2x faster**! See our [Phi-3 Medium notebook](https://colab.research.google.com/drive/1hhdhBa1j_hsymiW9m-WzxQtgqTH_NHqi?usp=sharing)
10. [**NEW**] We make Gemma-2 9b / 27b **2x faster**! See our [Gemma-2 9b notebook](https://colab.research.google.com/drive/1vIrqH5uYDQwsJ4-OO3DErvuv4pBgVwk4?usp=sharing)
11. [**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)
12. [**NEW**] We make Mistral NeMo 12B 2x faster and fit in under 12GB of VRAM! [Mistral NeMo notebook](https://colab.research.google.com/drive/17d3U-CAIwzmbDRqbZ9NnpHxCkmXB6LZ0?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>